In [1]:
using Random
using Revise

using BayesianExperiments
Revise.track(BayesianExperiments)

using Distributions: pdf, cdf
using ProgressMeter: @showprogress

┌ Info: Precompiling BayesianExperiments [0200d6b2-355c-4cc6-8967-0ef505c43a9c]
└ @ Base loading.jl:1278
┌ Warning: Package BayesianExperiments does not have StaticArrays in its dependencies:
│ - If you have BayesianExperiments checked out for development and have
│   added StaticArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with BayesianExperiments
└ Loading StaticArrays into BayesianExperiments from project dependency, future warnings for BayesianExperiments are suppressed.


In [6]:
Random.seed!(12)
data1 = rand(Normal(0.01, 1), 100_000)
data2 = rand(Normal(0.01, 1), 100_000);

In [5]:
s1 = NormalStatistics(data1)
s2 = NormalStatistics(data2)

s_split = update!(s1, s2)

NormalStatistics(200000, 0.008911898952613757, 1.0013483183212577)

In [6]:
s_split

NormalStatistics(200000, 0.008911898952613757, 1.0013483183212577)

In [7]:
NormalStatistics([data1;data2])

NormalStatistics(200000, 0.008911898952613757, 1.001345814936292)

In [8]:
function calc_bf(data)
    stats = NormalStatistics(data)

    model = EffectSizeModel(0.0, 0.1)
    stoppingrule = BayesFactorThresh(5)
    experiment = ExperimentBF(
        model=model, p0=0.5, rule=stoppingrule, stats=stats)

    bf = metrics(experiment)
    
    return bf
end

calc_bf (generic function with 1 method)

In [9]:
calc_bf(data1)

1.0354127801663817

In [10]:
calc_bf(data2)

2.714478677744355

In [11]:
calc_bf([data1;data2])

61.32577983782878

In [12]:
stats = NormalStatistics(data1)

model = EffectSizeModel(0.0, 0.1)
stoppingrule = BayesFactorThresh(5)
experiment = ExperimentBF(
    model=model, p0=0.5, rule=stoppingrule, stats=stats)

bf = metrics(experiment)

1.0354127801663817

In [13]:
decide!(experiment)

false

In [14]:
model = EffectSizeModel(0.0, 0.5)

EffectSizeModel(0.0, 0.5)

In [15]:
?NormalStatistics

search: NormalStatistics LogNormalStatistics



No documentation found.

# Summary

```
struct NormalStatistics <: BayesianExperiments.ModelStatistics
```

# Fields

```
n     :: Int64
meanx :: Real
sdx   :: Real
```

# Supertype Hierarchy

```
NormalStatistics <: BayesianExperiments.ModelStatistics <: Any
```


In [16]:
?EffectSizeModel

search: EffectSizeModel



```
EffectSizeModel <: ProbabilisticModel
```

A standard effect size model has two hypotheses: $H_1$(null) an $H_2$(alternative):

1. $H_1$: $\mu = m_0$
2. $H_2$: $\mu ≠ m_0$

with the population mean $\mu$ and pre-specified standard deviation $\sigma$. We want to test whether  $\mu$ is equal to $m_0$ or not.

The prior of the standard effect size is 

$\delta | H_2 \sim \text{Normal}(0, n_0)$

where $\delta$ is the standard effect size and $n_0$ can be considered as a prior sample size.  The standard effect size $\delta$ is defined as 

$\delta = \frac{\mu - m_0}{\sigma}.$

In practice, the standard deviations are unknown but in large sample scenario we assume  they are known and use their estimates.

## Fileds

  * `μ0`: mean of null hypothesis
  * `n0`: Prior sample size. `1/n0` is the prior standard deviation.

## References

1. [Chapter 5 hypothesis Testing with Normal Populations](  https://statswithr.github.io/book/hypothesis-testing-with-normal-populations.html)   in *An Introduction to Bayesian Thinking*.
2. Deng, Alex, Jiannan Lu, and Shouyuan Chen. "Continuous monitoring of A/B tests without pain:  Optional stopping in Bayesian testing." 2016 IEEE international conference on data science  and advanced analytics (DSAA). IEEE, 2016.


In [18]:
@show n0 = 32.7^2
m0 = 0.5
x̄ = 0.500177
σ = 0.5
n = 1.0449e8;
σ0 = 1/sqrt(n0)

stats = NormalStatistics(n=1.0449e8, meanx=x̄, sdx=σ)
@show δ = effectsize(stats, μ0=m0)
@show z = δ*sqrt(n)
@show sqrt(σ0^2+1/n)
@show sqrt(1/n)

n0 = 32.7 ^ 2 = 1069.2900000000002
δ = effectsize(stats, μ0 = m0) = 0.00035399999999996545
z = δ * sqrt(n) = 3.618600397943581
sqrt(σ0 ^ 2 + 1 / n) = 0.030581196229255123
sqrt(1 / n) = 9.782787848062487e-5


9.782787848062487e-5

These two formula is identical:

In [19]:
bf12 = sqrt((n+n0)/n0)*exp(-0.5*n/(n+n0)*n*δ^2)
bf21 = 1/bf12

@show bf21
@show bf12 = 1/ bf21;

bf21 = 2.2303006941904915
bf12 = 1 / bf21 = 0.44837003485889126


In [20]:
model = EffectSizeModel(m0, σ0)

EffectSizeModel(0.5, 0.030581039755351678)

In [21]:
bayesfactor(model, stats)

2.2303006941904906

In [22]:
bf2(es, n, n0) = 
    sqrt((n+n0)/n0)*exp(-0.5*n/(n+n0)*n*es^2)

bf2 (generic function with 1 method)

In [23]:
@show bf12  = bf2(δ, n, n0)
@show bf21 = 1/bf12

bf12 = bf2(δ, n, n0) = 0.44837003485889126
bf21 = 1 / bf12 = 2.2303006941904915


2.2303006941904915

## FDR control

In [58]:
function sample_bf_composite(δs, n, ns; σ=1, σ0=0.1, thresh=9)
    numruns = []
    rejections = []
    model = EffectSizeModel(0.0, σ0)
    @showprogress for δ in δs
        i = 100
        xs = rand(Normal(δ, 1), n)
        stoppingrule = BayesFactorThresh(thresh)
        experiment = ExperimentBF(model=model, rule=stoppingrule)
        while (i < n) && (experiment.rejection==false)
            i += 1
            stats = NormalStatistics(xs[1:i])
#             stats = NormalStatistics(meanx = mean(xs[1:i]), sdx=σ, n=i)
            #update!(experiment, stats)
            experiment.stats = stats
            decide!(experiment)
        end
        push!(numruns, i)
        push!(rejections, experiment.rejection)
    end
    numruns, rejections
end

sample_bf_composite (generic function with 1 method)

In [1]:
n  = 1000
ns = 5000
thresh = 9
σ0 = 0.1

numruns1, rej1 = sample_bf_composite(zeros(ns), n, ns, thresh=thresh);
deltas = rand(Normal(0, 0.1), ns);
numruns2, rej2 = sample_bf_composite(deltas, n, ns, thresh=thresh);

LoadError: UndefVarError: sample_bf_composite not defined

In [60]:
@show num_false_dis = sum(rej1 .=== true)
@show num_true_dis  = sum(rej2 .=== true)

println("\nType I Error: ", num_false_dis / ns)
println("Power       : ",   num_true_dis / ns)
println("FDR         : ",   num_false_dis / (num_true_dis + num_false_dis))

num_false_dis = sum(rej1 .=== true) = 263
num_true_dis = sum(rej2 .=== true) = 2514

Type I Error: 0.0526
Power       : 0.5028
FDR         : 0.094706517824991


In [61]:
xs = rand(Normal(0.0, 1), 1000)

bfs = []
for i in 2:1000
    stats = NormalStatistics(xs[1:i])
    update!(experiment, stats)
    push!(bfs, bayesfactor(experiment))
    decide!(experient)
end

┌ Error: Failed to revise /Users/jiangtao.fu/Downloads/BayesianExperiments.jl/src/model.jl
│   exception = Revise.ReviseEvalException("/Users/jiangtao.fu/Downloads/BayesianExperiments.jl/examples/none:0", ErrorException("invalid redefinition of constant EffectSizeModel"), Any[(top-level scope at none:0, 1)])
└ @ Revise /Users/jiangtao.fu/.julia/packages/Revise/moD4B/src/packagedef.jl:707
┌ Warning: The running code does not match the saved version for the following files:
│ 
│   /Users/jiangtao.fu/Downloads/BayesianExperiments.jl/src/model.jl
│ 
│ If the error was due to evaluation order, it can sometimes be resolved by calling `Revise.retry()`.
│ Use Revise.errors() to report errors again. Only the first error in each file is shown.
│ Your prompt color may be yellow until the errors are resolved.
└ @ Revise /Users/jiangtao.fu/.julia/packages/Revise/moD4B/src/packagedef.jl:805


LoadError: MethodError: no method matching bayesfactor(::EffectSizeModel, ::NormalStatistics)

In [2]:
gradesA = [55, 65, 65, 68, 70]
gradesB = [56, 60, 62, 66]

4-element Array{Int64,1}:
 56
 60
 62
 66

In [3]:
twosamplestats = TwoSampleStatistics(NormalStatistics(gradesA), NormalStatistics(gradesB))

2-element StaticArrays.SArray{Tuple{2},NormalStatistics,1,2} with indices SOneTo(2):
 NormalStatistics(5, 64.6, 5.770615218501404)
 NormalStatistics(4, 61.0, 4.163331998932265)

In [5]:
effectsize(twosamplestats)

0.6998920003079266

In [6]:
stats = TwoSampleStatistics(
    NormalStatistics(meanx=28.8, sdx=13.5, n=133),
    NormalStatistics(meanx=30.6, sdx=14.3, n=867)
)

2-element StaticArrays.SArray{Tuple{2},NormalStatistics,1,2} with indices SOneTo(2):
 NormalStatistics(133, 28.8, 13.5)
 NormalStatistics(867, 30.6, 14.3)

In [7]:
effectsize(stats)

-0.1267893536694397

In [8]:
?bayesfactor

search: bayesfactor BayesFactorThresh



No documentation found.

`BayesianExperiments.bayesfactor` is a `Function`.

```
# 2 methods for generic function "bayesfactor":
[1] bayesfactor(model::EffectSizeModel, stats::NormalStatistics) in BayesianExperiments at /Users/jiangtao.fu/Downloads/BayesianExperiments.jl/src/model.jl:315
[2] bayesfactor(experiment::ExperimentBF) in BayesianExperiments at /Users/jiangtao.fu/Downloads/BayesianExperiments.jl/src/experiment.jl:129
```


In [ ]:

"""
## References

- https://www.jmp.com/en_ch/statistics-knowledge-portal/t-test/two-sample-t-test.html
- https://en.wikipedia.org/wiki/Student%27s_t-test#Independent_two-sample_t-test
"""
function tstat_twosample_pooled(x̄1, x̄2, sd1, sd2, n1, n2)
    es = effectsize(x̄1 - x̄2, pooledsd(sd1, sd2, n1, n2))
    ess = effsamplesize(n1, n2)
    es * sqrt(ess)
end

function tstat_twosample_welch(x̄1, x̄2, sd1, sd2, n1, n2)
    se = sqrt(sd1^2/n1 + sd2^2/n2)
    (x̄1 - x̄2) / se
end

function bayesfactor_ttest(t, v, n; r=0.707, rtol=1e-8)
    numerator = (1 + t^2/v)^(-(v+1)/2)
    denominator, _ = quadgk(
        g -> (1+n*g*r^2)^(-0.5) * (1+t^2/((1+n*g*r^2)*v))^(-(v+1)/2) 
            * (2π)^(-0.5) * g^(-1.5) * exp(-1/(2*g)),
        0, Inf,
        rtol=rtol)
  # we take invserse to get b_1_0, instead of b_0_1
  denominator/numerator
end